In [ ]:
'''uncomment when you need dataset'''

In [2]:
import numpy as np
import pandas as pd
import pickle
from datetime import timedelta
import sys
sys.path.append('/Users/leilei/Documents/DS1011Project/Project1011_Mortality_Predcition/data_preprocessing')
#from data_cleaning import word_tokenize_by_string
import itertools

In [3]:
def data_split(patient_list, downsampling_rate = 0.3):
    # Argments:
    #       patient_list : in format [[patient_id, label]] where death label = 1 otherwise label = 0
    #       downsampling_rate : the percentage of negative sample in the final training data
    # Output:
    #        train_data, validation data and test_data
    
    while True:
        
        np.random.rand(patient_list.shape[0]).argsort()
        np.take(patient_list,np.random.rand(patient_list.shape[0]).argsort(),axis=0,out=patient_list)
        patient_list = np.array(patient_list)
        num_patients = len(patient_list[:,0])
        train_data = patient_list[:int(0.7*num_patients)]
        val_data   = patient_list[int(0.7*num_patients):int(0.8*num_patients)]
        test_data  = patient_list[int(0.8*num_patients):num_patients]
        
        #downsampling
        if sum(train_data[:,1]) > 10:
            break
        else:
            np.random.rand(patient_list.shape[0]).argsort()
            np.take(patient_list,np.random.rand(patient_list.shape[0]).argsort(),axis=0,out=patient_list)
    
    if sum(train_data[:,1])/len(train_data[:,1]) <= 0.3:

        downsampling_size = int(sum(train_data[:,1])*(1 - downsampling_rate)/downsampling_rate)

        train_data_survive = train_data[train_data[:,1] != 1][:downsampling_size]
        train_data_dead = train_data[train_data[:,1] == 1]
        train_data = np.vstack((train_data_survive,train_data_dead))
        #random.shuffle(train_data)
        np.random.rand(train_data.shape[0]).argsort()
        np.take(train_data,np.random.rand(train_data.shape[0]).argsort(),axis=0,out=train_data)
        print('The percentage of negative sample after downsampling is {:.1%}'.format(sum(train_data[:,1])/len(train_data[:,1])))
        return train_data[:,0], val_data[:,0], test_data[:,0]

    else:
        print('The percentage of negative sample is {:.1%}'.format(sum(train_data[:,1])/len(train_data[:,1])))
        return train_data[:,0], val_data[:,0], test_data[:,0]

### read_in dataset

In [4]:
label = np.load('./data/label.npy')
mortality = np.load('./data/mortality_label.npy')
patient_timesheet = pd.read_pickle('./data/patient_timesheet_final.pickle')
notes_cleaned_pickle = pd.read_csv('./data/cleaned_notes.csv')

CHARTTIME = notes_cleaned_pickle['CHARTTIME']
#DOCUMENTS = notes_cleaned_pickle['DOCUMENTS']
SUBJECT_ID = notes_cleaned_pickle['SUBJECT_ID']

CHARTTIME = [np.datetime64(x, 'ns') for x in CHARTTIME ]
#dictionary_for_notes = dict(zip(keys_for_notes, DOCUMENTS))

keys_for_notes = tuple(zip(SUBJECT_ID, CHARTTIME))

id_inhospital_deathtime = pd.read_csv('./data/id_inhospital_deathtime.csv', index_col = 1)
id_inhospital_deathtime = id_inhospital_deathtime[id_inhospital_deathtime.index.isin(SUBJECT_ID)]
mortality_label= ~id_inhospital_deathtime.DEATHTIME.isnull()*1
#index is patient and label is dead or not

In [6]:
patient_list = np.array(list(zip(list(mortality_label.index), list(mortality_label))))

In [7]:
label_ = label.copy()
mortality_ = mortality.copy()
patient_timesheet_ = patient_timesheet.copy()
notes_cleaned_pickle_ = notes_cleaned_pickle.copy()
mortality_label_ = mortality_label.copy()

In [16]:
import random
train_sub, val_sub, test_sub = data_split(patient_list)
len(train_sub), len(val_sub), len(test_sub),set(train_sub).intersection(val_sub), set(test_sub).intersection(val_sub),set(train_sub).intersection(test_sub)

The percentage of negative sample after downsampling is 30.0%


(11010, 3487, 6975, set(), set(), set())

In [11]:
len(patient_list)

34872

In [12]:
voca=label.item()['WORD_LIST']

In [8]:
def data_selection(timesheet, time_period, lower_bound = np.timedelta64(0, 'm')):
    selected_notes = []
    for sub in timesheet.keys():
        i = 0
        try:
            for interval in timesheet[sub]['CHARTTIME_interval']:
                if interval<=time_period:
                    selected_notes.append((sub, timesheet[sub]['CHARTTIME'][i]))
                i+=1
        except KeyError:
            print(sub)
         
    return selected_notes

In [9]:
selected_notes_15 = data_selection(patient_timesheet, np.timedelta64(15, 'm'))
selected_notes_15 = sorted(selected_notes_15, key = lambda i: (i[0],i[1]))

selected_notes_6 = data_selection(patient_timesheet, np.timedelta64(6, 'h'))
selected_notes_6 = sorted(selected_notes_6, key = lambda i: (i[0],i[1]))

selected_notes_12 = data_selection(patient_timesheet, np.timedelta64(12, 'h'))
selected_notes_12 = sorted(selected_notes_12, key = lambda i: (i[0],i[1]))

selected_notes_24 = data_selection(patient_timesheet, np.timedelta64(24, 'h'))
selected_notes_24 = sorted(selected_notes_24, key = lambda i: (i[0],i[1]))

532
1687
2601
4108
4632
5988
7119
8338
9642
11779
11983
12650
13567
15972
17579
18663
20131
21059
22745
22940
23739
24605
24621
25365
92001
28078
32705
77805
57133
532
1687
2601
4108
4632
5988
7119
8338
9642
11779
11983
12650
13567
15972
17579
18663
20131
21059
22745
22940
23739
24605
24621
25365
92001
28078
32705
77805
57133
532
1687
2601
4108
4632
5988
7119
8338
9642
11779
11983
12650
13567
15972
17579
18663
20131
21059
22745
22940
23739
24605
24621
25365
92001
28078
32705
77805
57133
532
1687
2601
4108
4632
5988
7119
8338
9642
11779
11983
12650
13567
15972
17579
18663
20131
21059
22745
22940
23739
24605
24621
25365
92001
28078
32705
77805
57133


In [ ]:
selected_notes_12[:5]

In [10]:
sub_id_dict_selected = {}
sub_id_dict_selected['15min'] = selected_notes_15
sub_id_dict_selected['24h'] = selected_notes_24
sub_id_dict_selected['6h'] = selected_notes_6
sub_id_dict_selected['12h'] = selected_notes_12

In [11]:
pickle.dump(sub_id_dict_selected, open( "./data/subID_notes_selected_Dec1.p", "wb" ) )

In [19]:
len(selected_notes_15),len(selected_notes_6), len(selected_notes_12), len(selected_notes_24)
#15min, 6h, 12h, 24h

(1193, 31728, 61259, 112557)

In [ ]:
#patient_id, the notes time
selected_notes[:2]

In [ ]:
### It has been Organized in function get_splitted_index in selecting and matching dataset

In [41]:
#15min
table_selected_notes.shape , table_selected_notes.sub_id.nunique() 

((1193, 2), 1116)

In [70]:
#6h
table_selected_notes.sub_id.nunique(),table_selected_notes.shape 

(19121, (31728, 2))

In [84]:
#12h
table_selected_notes.sub_id.nunique(),table_selected_notes.shape #91344

(26685, (61259, 2))

In [97]:
#24h
table_selected_notes.sub_id.nunique(),table_selected_notes.shape #142642

(30388, (112557, 2))

In [23]:
len(table_selected),len(SUBJECT_ID)

(1116, 1280007)

### taking notes

In [ ]:
for key in dictionary_for_notes.keys():
    if type(dictionary_for_notes[key])==float:
        dictionary_for_notes[key] = ' '

In [12]:
def notes_take(dictionary_for_notes, table_selecte):
    '''
    dictionary_for_notes: (subject_id, chart-time) documents_content
    table_selected: pandas series with sub_id as its index and list of creation time stamps for notes
    return: notes_selected_by_sub: concatnated notes for patients, summ: which is the no. of wrong keys
    '''
    notes_selected_by_sub = []
    label = []
    summ = 0
    missed=[]
    for sub in table_selecte.index:
        notes = []
        try:
            for i in range(len(table_selecte[sub])):
                try:
                    notes.append(dictionary_for_notes[(sub, table_selecte[sub][i])])
                    #print((sub, table_selecte[sub][i]))
                except KeyError:
                    #print((sub, table_selecte[sub][i]))
                    missed.append((sub, table_selecte[sub][i]))
                    summ+=1
            notes_selected_by_sub.append([' '.join(notes), sub])
        except TypeError:
            print(notes)
            
    return notes_selected_by_sub, summ, missed

In [17]:
notes_selected_by_sub, summ, missed = notes_take(dictionary_for_notes,table_selected )

NameError: name 'dictionary_for_notes' is not defined

In [ ]:
#the ids which need to be selected
table_selected_notes.head()

In [ ]:
table_selected_notes.shape

In [ ]:
notes_selected_by_sub = pd.DataFrame(notes_selected_by_sub)
notes_selected_by_sub.columns = ['notes','sub_id']

In [ ]:
notes_selected_by_sub.head()

In [ ]:
voca_set = set(voca)
def filter_by_set(notes_list):
    '''
    notes_list: each note in a list
    voca: the dictionary we use
    return: filtered notes
    '''
    notes_set = set(notes_list)
    notes_vaild = notes_set & voca_set
    new_li=[i for i in notes_list if i in notes_vaild]
    return new_li

def filter_word(notes_list, voca):
    '''
    notes_list: each note in a list
    voca: the dictionary we use
    return: filtered notes
    '''
    new_li=[i for i in notes_list if i in voca]
    return new_li

In [ ]:
notes_selected_by_sub['notes'] = notes_selected_by_sub['notes'].map(lambda x: word_tokenize_by_string(x))

In [ ]:
notes_selected_by_sub.head()

In [ ]:
notes_after = notes_selected_by_sub['notes'].map(lambda x: filter_by_set(x))
notes_selected_by_sub['notes']=notes_after

In [ ]:
length_after = [len(x) for x in notes_after]
length_before = [len(x) for x in notes_selected_by_sub['notes']]

sum(np.array(length_before)  != np.array(length_after))

--------

### Join tokenized notes with Label

In [98]:
table_selected_notes.head()

,creation_time,sub_id
0,2101-10-20 22:23:00,3
1,2101-10-21 01:00:00,3
2,2101-10-21 06:10:00,3
3,2101-10-21 06:58:00,3
4,2101-10-21 14:27:00,3


In [100]:
len(mortality_label)

34872

In [103]:
merged.shape

(30388, 3)

In [104]:
merged.head()

,sub_id,creation_time,DEATHTIME
0,3,"[2101-10-20T22:23:00.000000000, 2101-10-21T01:...",__label__LIVE
1,4,"[2191-03-16T05:01:00.000000000, 2191-03-16T12:...",__label__LIVE
2,6,"[2175-05-30T16:25:00.000000000, 2175-05-30T17:...",__label__LIVE
3,9,"[2149-11-09T17:53:00.000000000, 2149-11-10T04:...",__label__DEAD
4,11,"[2178-04-16T14:49:00.000000000, 2178-04-16T15:...",__label__LIVE


In [105]:
merged.shape, len(set(merged.sub_id))

((30388, 3), 30388)

In [55]:
#15min WRONG!!!
merged['DEATHTIME'].value_counts()

__label__LIVE    16335
__label__DEAD     2786
Name: DEATHTIME, dtype: int64

In [78]:
#6h
merged['DEATHTIME'].value_counts()

__label__LIVE    16335
__label__DEAD     2786
Name: DEATHTIME, dtype: int64

In [92]:
#12h
merged['DEATHTIME'].value_counts()

__label__LIVE    23080
__label__DEAD     3605
Name: DEATHTIME, dtype: int64

In [ ]:
#24h
merged['DEATHTIME'].value_counts()

In [14]:
SAVE_PATH = './data/'
import os
''' 
def save_splited_data(train_index, val_index,test_index, data, save_file):
    
    f_train = open(os.path.join(SAVE_PATH, 'train_'+save_file+'.txt'),"w")
    f_val = open(os.path.join(SAVE_PATH, 'val_'+save_file+'.txt'),"w")
    f_test = open(os.path.join(SAVE_PATH, 'test_'+save_file+'.txt'),"w")

    for i in range(merged.shape[0]):
        row = merged.iloc[i,:]
        if row['sub_id'] in train_index:
            f_train.write(' '.join(row['notes'])+' '+row['DEATHTIME']+'\n')
        elif row['sub_id'] in val_index:
            f_val.write(' '.join(row['notes'])+' '+row['DEATHTIME']+'\n')
        else:
            f_test.write(' '.join(row['notes'])+' '+row['DEATHTIME']+'\n')
'''

        
def split_data_by_time(train_sub,val_sub,test_sub, sub_id ):
    train_sub_t = [x for x in sub_id if x in train_sub]
    val_sub_t = [x for x in sub_id if x in val_sub]
    test_sub_t = [x for x in sub_id if x in test_sub]
    return train_sub_t,val_sub_t, test_sub_t

In [18]:
train_sub_15,val_sub_15, test_sub_15 = split_data_by_time(train_sub, val_sub, test_sub, merged.sub_id)
train_sub_6,val_sub_6, test_sub_6 = split_data_by_time(train_sub, val_sub, test_sub, merged.sub_id)
train_sub_12,val_sub_12, test_sub_12 = split_data_by_time(train_sub, val_sub, test_sub, merged.sub_id)
train_sub_24,val_sub_24, test_sub_24 = split_data_by_time(train_sub, val_sub, test_sub, merged.sub_id)

NameError: name 'merged' is not defined

In [158]:
def get_splitted_index(train_sub, val_sub, test_sub, selected_notes):
    '''
    train_sub: the training index
    val_sub: the validation index
    test_sub: the test index
    selected_notes: the index of notes selected for certain period
    return: patient index for train, val and test set
    '''
    sub_id = [x[0] for x in selected_notes]
    creation_time = [x[1] for x in selected_notes]
    table_selected_notes = pd.DataFrame({'sub_id': sub_id, 'creation_time':creation_time})
    table_selected = table_selected_notes.groupby(['sub_id'])['creation_time'].unique()

    table_selected = pd.DataFrame(table_selected).reset_index()

    merged = table_selected.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)
    #merged = notes_selected_by_sub.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)

    merged['DEATHTIME'] = merged['DEATHTIME'].map(lambda x: '__label__DEAD' if x == 1 else '__label__LIVE')
    
    new_train, new_val, new_test = split_data_by_time(train_sub, val_sub, test_sub, sub_id)
    return new_train, new_val, new_test

In [ ]:
#save_splited_data(train_sub_6,val_sub_6,test_sub_6, merged,'6h')

In [ ]:
TOBERUNNED

In [107]:
len(train_sub_15), len(val_sub_15), len(test_sub_15), sum([len(train_sub_15), len(val_sub_15), len(test_sub_15)])

(352, 116, 229, 697)

In [108]:
len(train_sub_6), len(val_sub_6), len(test_sub_6), sum([len(train_sub_6), len(val_sub_6), len(test_sub_6)])

(6118, 1904, 3896, 11918)

In [109]:
len(train_sub_12), len(val_sub_12), len(test_sub_12), sum([len(train_sub_12), len(val_sub_12), len(test_sub_12)])

(8369, 2670, 5371, 16410)

In [110]:
len(train_sub_24), len(val_sub_24), len(test_sub_24), sum([len(train_sub_24), len(val_sub_24), len(test_sub_24)])

(9562, 3055, 6067, 18684)

In [111]:
set(train_sub_15).intersection(val_sub_15), set(test_sub_15).intersection(val_sub_15),\
set(train_sub_15).intersection(test_sub_15)

(set(), set(), set())

In [112]:
set(train_sub_6).intersection(val_sub_6), set(test_sub_6).intersection(val_sub_6),\
set(train_sub_6).intersection(test_sub_6)

(set(), set(), set())

In [113]:
set(train_sub_12).intersection(val_sub_12), set(test_sub_12).intersection(val_sub_6),\
set(train_sub_12).intersection(test_sub_12)

(set(), set(), set())

In [114]:
set(train_sub_24).intersection(val_sub_24), set(test_sub_24).intersection(val_sub_24),\
set(train_sub_24).intersection(test_sub_24)

(set(), set(), set())

In [115]:
sub_id_dict = {}
time_list = ['15m', '24h', '6h',' 12h']
sub_id_dict['24h'] = {}
sub_id_dict['6h'] = {}
sub_id_dict['12h'] = {}
sub_id_dict['15m'] = {}

sub_id_dict['24h']['train_24'] = train_sub_24
sub_id_dict['24h']['val_24'] = val_sub_24
sub_id_dict['24h']['test_24'] = test_sub_24

sub_id_dict['12h']['train_12'] = train_sub_12
sub_id_dict['12h']['val_12'] = val_sub_12
sub_id_dict['12h']['test_12'] = test_sub_12

sub_id_dict['6h']['train_6'] = train_sub_6
sub_id_dict['6h']['val_6'] = val_sub_6
sub_id_dict['6h']['test_6'] = test_sub_6

sub_id_dict['15m']['train_15'] = train_sub_15
sub_id_dict['15m']['val_15'] = val_sub_15
sub_id_dict['15m']['test_15'] = test_sub_15

#sub_id_dict['all']['train'] = train_sub

In [125]:
#pickle.dump(sub_id_dict, open( "patient_id_splited_data.p", "wb" ) )
pickle.dump(sub_id_dict, open( "./data/patient_id_splited_data_Nov28.p", "wb" ) )
pickle.dump(train_sub, open( "./data/patient_id_splited_all_train_Nov28.p", "wb" ) )
pickle.dump(val_sub, open( "./data/patient_id_splited_all_val_Nov28.p", "wb" ) )
pickle.dump(test_sub, open( "./data/patient_id_splited_all_test_Nov28.p", "wb" ) )

In [126]:
train_sub

array([42261, 22496, 13603, ...,  9310, 52010, 10594], dtype=int64)

In [119]:
train_idx_24 = pd.DataFrame(train_sub_24)
train_idx_24.columns = ['sub_id']
train_idx_24_merged = train_idx_24.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)
train_idx_24_merged['DEATHTIME'] = train_idx_24_merged['DEATHTIME'].map(lambda x: '__label__DEAD' if x == 1 else '__label__LIVE')
train_idx_24_merged['DEATHTIME'].value_counts()

__label__LIVE    6746
__label__DEAD    2816
Name: DEATHTIME, dtype: int64

In [120]:
train_idx_12 = pd.DataFrame(train_sub_12)
train_idx_12.columns = ['sub_id']
train_idx_12_merged = train_idx_12.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)
train_idx_12_merged['DEATHTIME'] = train_idx_12_merged['DEATHTIME'].map(lambda x: '__label__DEAD' if x == 1 else '__label__LIVE')
train_idx_12_merged['DEATHTIME'].value_counts()

__label__LIVE    5882
__label__DEAD    2487
Name: DEATHTIME, dtype: int64

In [121]:
train_idx_15 = pd.DataFrame(train_sub_15)
train_idx_15.columns = ['sub_id']
train_idx_15_merged = train_idx_15.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)
train_idx_15_merged['DEATHTIME'] = train_idx_15_merged['DEATHTIME'].map(lambda x: '__label__DEAD' if x == 1 else '__label__LIVE')
train_idx_15_merged['DEATHTIME'].value_counts()

__label__LIVE    232
__label__DEAD    120
Name: DEATHTIME, dtype: int64

In [122]:
train_idx_6 = pd.DataFrame(train_sub_6)
train_idx_6.columns = ['sub_id']
train_idx_6_merged = train_idx_6.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)
train_idx_6_merged['DEATHTIME'] = train_idx_6_merged['DEATHTIME'].map(lambda x: '__label__DEAD' if x == 1 else '__label__LIVE')
train_idx_6_merged['DEATHTIME'].value_counts()

__label__LIVE    4208
__label__DEAD    1910
Name: DEATHTIME, dtype: int64

In [123]:
train_idx = pd.DataFrame(train_sub)
train_idx.columns = ['sub_id']
train_idx_merged = train_idx.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)
train_idx_merged['DEATHTIME'] = train_idx_merged['DEATHTIME'].map(lambda x: '__label__DEAD' if x == 1 else '__label__LIVE')
train_idx_merged['DEATHTIME'].value_counts()

__label__LIVE    7725
__label__DEAD    3311
Name: DEATHTIME, dtype: int64

In [ ]:
import pickle
sub_id_dict_check = pickle.load( open("patient_id_splited_data_Nov28.p", "rb" ) )

In [ ]:
for time in sub_id_dict_check.keys():
    key_li = list(sub_id_dict_check[time].keys())
    tobe_checked_li = list(itertools.combinations(key_li,2))
    for item in tobe_checked_li:
        print(set(sub_id_dict_check[time][item[0]]).intersection(sub_id_dict_check[time][item[1]]))
        #if len(set(sub_id_dict_check[time][item[0]]).intersection(sub_id_dict_check[time][item[1]])) != 0:
            #print('overlap')
#so no overlap at all

In [ ]:
'''
with open('notes_handled_test.txt','w') as f:
    for i in range(merged.shape[0]):
        row = merged.iloc[i,:]
        f.write(' '.join(row['notes'])+' '+row['DEATHTIME']+'\n')

with open('notes_handled_15min.txt','w') as f:
    for i in range(merged.shape[0]):
        row = merged.iloc[i,:]
        f.write(' '.join(row['notes'])+'\t'+'{}'.format(row['sub_id'])+'\t'+row['DEATHTIME']+'\n')
        
'''

### Generate Not Downsampling
1. put the rest from patient list to training sub
<br>
2. re-select and save

In [129]:
sub_id_dict

{'12h': {'test_12': [4,
   9,
   25,
   38,
   41,
   52,
   53,
   71,
   75,
   77,
   80,
   88,
   91,
   97,
   134,
   138,
   147,
   154,
   164,
   176,
   202,
   203,
   208,
   212,
   214,
   218,
   223,
   231,
   240,
   250,
   272,
   286,
   307,
   313,
   314,
   351,
   352,
   360,
   364,
   373,
   376,
   377,
   396,
   400,
   413,
   414,
   415,
   417,
   440,
   441,
   470,
   481,
   489,
   490,
   505,
   510,
   521,
   534,
   536,
   544,
   561,
   564,
   568,
   571,
   594,
   628,
   634,
   654,
   665,
   671,
   689,
   705,
   712,
   738,
   744,
   745,
   746,
   755,
   773,
   778,
   791,
   810,
   840,
   854,
   859,
   881,
   899,
   907,
   915,
   931,
   932,
   937,
   938,
   940,
   954,
   956,
   961,
   970,
   975,
   976,
   984,
   1003,
   1004,
   1005,
   1016,
   1034,
   1042,
   1045,
   1060,
   1062,
   1081,
   1102,
   1103,
   1107,
   1108,
   1124,
   1125,
   1127,
   1135,
   1148,
   1151,
   1158,
 

In [136]:
len(train_sub), len(val_sub), len(test_sub)

(11036, 3487, 6975)

In [137]:
len(set(train_sub)), len(set(val_sub)), len(set(test_sub))

(11036, 3487, 6975)

In [148]:
#which is 
existing = list(train_sub) + list(val_sub) + list(test_sub)
new_training_sub = [i for i in list(mortality_label.index) if i not in existing]
new_training_sub = list(train_sub) + new_training_sub

In [155]:
len(new_training_sub), len(val_sub), len(test_sub), len(patient_list)==sum([len(new_training_sub), len(val_sub), len(test_sub)])

(24410, 3487, 6975, True)

In [24]:
def split_data_by_time(train_sub,val_sub,test_sub, sub_id ):
    train_sub_t = [x for x in sub_id if x in train_sub]
    val_sub_t = [x for x in sub_id if x in val_sub]
    test_sub_t = [x for x in sub_id if x in test_sub]
    return train_sub_t,val_sub_t, test_sub_t

def get_splitted_index(train_sub, val_sub, test_sub, selected_notes):
    '''
    train_sub: the training index
    val_sub: the validation index
    test_sub: the test index
    selected_notes: the index of notes selected for certain period
    return: patient index for train, val and test set
    '''
    sub_id = [x[0] for x in selected_notes]
    creation_time = [x[1] for x in selected_notes]
    table_selected_notes = pd.DataFrame({'sub_id': sub_id, 'creation_time':creation_time})
    table_selected = table_selected_notes.groupby(['sub_id'])['creation_time'].unique()
    print(table_selected.shape)
    table_selected = pd.DataFrame(table_selected).reset_index()

    merged = table_selected.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)
    #merged = notes_selected_by_sub.merge(pd.DataFrame(mortality_label), how= 'left', left_on='sub_id', right_index=True)
    print(merged.shape)
    merged['DEATHTIME'] = merged['DEATHTIME'].map(lambda x: '__label__DEAD' if x == 1 else '__label__LIVE')
    
    new_train, new_val, new_test = split_data_by_time(train_sub, val_sub, test_sub, merged.sub_id)
    return new_train, new_val, new_test

In [22]:
new_training_sub = train_sub

In [27]:
len(selected_notes_15)

31278

In [25]:
new_training_15, new_val_sub_15, new_test_sub_15 = \
get_splitted_index(new_training_sub, val_sub, test_sub, selected_notes_15)

new_training_6, new_val_sub_6, new_test_sub_6 = \
get_splitted_index(new_training_sub, val_sub, test_sub, selected_notes_6)

new_training_12,new_val_sub_12,new_test_sub_12 = \
get_splitted_index(new_training_sub, val_sub, test_sub, selected_notes_12)

new_training_24,new_val_sub_24,new_test_sub_24 = \
get_splitted_index(new_training_sub, val_sub, test_sub, selected_notes_24)

(15603,)
(15603, 3)
(25502,)
(25502, 3)
(29983,)
(29983, 3)
(32253,)
(32253, 3)


In [28]:
sub_id_dict_no_down = {}
sub_id_dict_no_down['24h'] = {}
sub_id_dict_no_down['6h'] = {}
sub_id_dict_no_down['12h'] = {}
sub_id_dict_no_down['15m'] = {}

sub_id_dict_no_down['24h']['train_24'] = new_training_24
sub_id_dict_no_down['24h']['val_24'] = new_val_sub_24
sub_id_dict_no_down['24h']['test_24'] = new_test_sub_24

sub_id_dict_no_down['12h']['train_12'] = new_training_12
sub_id_dict_no_down['12h']['val_12'] = new_val_sub_12
sub_id_dict_no_down['12h']['test_12'] = new_test_sub_12

sub_id_dict_no_down['6h']['train_6'] = new_training_6
sub_id_dict_no_down['6h']['val_6'] = new_val_sub_6
sub_id_dict_no_down['6h']['test_6'] = new_test_sub_6

sub_id_dict_no_down['15m']['train_15'] = new_training_15
sub_id_dict_no_down['15m']['val_15'] = new_val_sub_15
sub_id_dict_no_down['15m']['test_15'] = new_test_sub_15
#sub_id_dict['all']['train'] = train_sub

In [176]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

new_val_sub_24 ==val_sub_24
new_val_sub_12 == val_sub_12
new_val_sub_6 == val_sub_6
new_val_sub_6 == val_sub_6


new_test_sub_24 == test_sub_24
new_test_sub_12 == test_sub_12
new_test_sub_6 == test_sub_6
new_test_sub_6 == test_sub_6

True

True

True

True

True

True

True

True

In [177]:
len(new_val_sub_24),len(val_sub_24)

(3055, 3055)

In [29]:
#which actually produced down sampled sample. Cna't be bothered to change name
pickle.dump(sub_id_dict_no_down, open( "./data/patient_id_splited_data_Dec3.p", "wb" ) )

In [178]:
#pickle.dump(sub_id_dict, open( "patient_id_splited_data.p", "wb" ) )
pickle.dump(sub_id_dict_no_down, open( "./data/patient_id_splited_data_Nov28_no_down.p", "wb" ) )
pickle.dump(new_training_sub, open( "./data/patient_id_splited_all_train_Nov28_no_down.p", "wb" ) )
#pickle.dump(val_sub, open( "./data/patient_id_splited_all_val_Nov28.p", "wb" ) )
#pickle.dump(test_sub, open( "./data/patient_id_splited_all_test_Nov28.p", "wb" ) )

In [ ]:
#should check, the validation and test should be same as before